# Exploratory Data Analysis
Understanding The dataset to explore how the data is present in the database and if there is a need of creating some aggregated tables that can help with:

* Vendor Selection for profitability
* Product Pricing Optimazation

In [1]:
import pandas as pd
import sqlite3

In [2]:
# creating database connection
conn = sqlite3.connect('inventory.db')

In [3]:
#checking table present in the database
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type ='table'",conn)
tables

,name


In [4]:
pd.read_sql("select count(*) from purchases", conn)

DatabaseError: Execution failed on sql 'select count(*) from purchases': no such table: purchases

In [ ]:
for table in tables ['name']:
    print('_'*50, f'{table}','_'*50)
    print('count of records:', pd.read_sql(f"select count(*) as count from {table}", conn)['count'].values[0])

In [ ]:
for table in tables ['name']:
    print('_'*50, f'{table}','_'*50)
    print('count of records:', pd.read_sql(f"select count(*) as count from {table}", conn)['count'].values[0])
    display(pd.read_sql(f"select * from {table} limit 5", conn))

In [ ]:
purchases = pd.read_sql_query("select * from purchases where VendorNumber = 4466", conn)
purchases

In [ ]:
purchase_prices = pd.read_sql_query("select * from purchase_prices where VendorNumber = 4466", conn)
purchase_prices

In [ ]:
vendor_invoice = pd.read_sql_query("select * from vendor_invoice where VendorNumber = 4466", conn)
vendor_invoice

In [ ]:
sales = pd.read_sql_query("select * from sales where VendorNo = 4466", conn)
sales

In [ ]:
purchases.groupby(['Brand', 'PurchasePrice'])[['Quantity', 'Dollars']].sum()

In [ ]:
sales.groupby('Brand')[['SalesDollars', 'SalesPrice', 'SalesQuantity']].sum()

In [ ]:
freight_summary = pd.read_sql_query("""select VendorNumber, SUM(Freight) as FreightCost
From vendor_invoice
Group BY VendorNumber""", conn)

In [ ]:
freight_summary

In [ ]:
purchases.columns

In [ ]:
purchase_prices.columns

In [ ]:
pd.read_sql_query("""SELECT
    p.VendorNumber,
    p.VendorName,
    p.Brand,
    p.PurchasePrice,
    pp.Volume,
    pp.Price as ActualPrice,
    SUM(p.Quantity) as TotalPurchaseQuantity,
    SUM(p.Dollars) as TotalPurchaseDollars
    FROM purchases p
    JOIN purchase_prices pp
    ON p.Brand = pp.Brand
    Where p.PurchasePrice > 0
    GROUP BY p.VendorNumber, p.VendorName, p.Brand
    ORDER BY TotalPurchaseDollars""", conn)

In [ ]:
sales.columns

In [ ]:
pd.read_sql_query("""SELECT
    VendorNo,
    Brand,
    SUM(SalesDollars) as TotalSalesDollars,
    SUM(SalesPrice) as TotaSalesPrice,
    SUM(SalesQuantity) as TotalSalesQuantity,
    SUM(ExciseTax) as TotalExciseTax
    FROM sales
    GROUP BY VendorNo, Brand
    ORDER BY TotalSalesDollars""", conn)

In [ ]:
vendor_sales_summary = pd.read_sql_query("""WITH FreightSummary AS (
    SELECT
        Vendornumber,
        SUM(Freight) AS FreightCost
    FROM vendor_invoice
    GROUP BY VendorNumber
),

PurchaseSummary AS (
    SELECT
         p.VendorNumber,
         p.VendorName,
         p.Brand,
         p.Description,
         pp.PurchasePrice,
         pp.Price AS ActualPrice,
         pp.Volume,
         SUM(p.Quantity) AS TotalPurchaseQuantity,
         SUM(p.Dollars) AS TotalPurchaseDollars
    FROM purchases p
    JOIN purchase_prices pp
        ON p.Brand =pp.Brand
    WHERE p.PurchasePrice > 0
    GROUP BY p.VendorNumber, p.VendorName, p.Brand, p.Description, p.PurchasePrice, pp.Price, pp.Volume
),

SalesSummary As (
    SELECT
        VendorNo,
        Brand,
        SUM(SalesQuantity) AS TotalSalesQuantity,
        SUM(SalesDollars) AS TotalSalesDollars,
        SUM(SalesPrice) AS TotalSalesPrice,
        SUM(ExciseTax) AS TotalExciseTax
    FROM Sales
    GROUP BY VendorNo, Brand
)

SELECT
    ps.VendorNumber,
    ps.VendorName,
    ps.Brand,
    ps.Description,
    ps.PurchasePrice,
    ps.ActualPrice,
    ps.Volume,
    ps.TotalPurchaseQuantity,
    ps.TotalPurchaseDollars,
    ss.TotalSalesQuantity,
    ss.TotalSalesDollars,
    ss.TotalSalesPrice,
    ss.TotalExciseTax,
    fs.FreightCost
FROM PurchaseSummary ps
LEFT JOIN SalesSummary ss
    ON ps.VendorNumber = ss.VendorNo
    AND ps.Brand = ss.Brand
LEFT JOIN FreightSummary fs
    ON ps.VendorNumber = fs.VendorNumber
ORDER BY ps.TotalPurchaseDollars DESC""", conn)

In [ ]:
vendor_sales_summary

In [ ]:
vendor_sales_summary.dtypes

In [ ]:
vendor_sales_summary.isnull().sum()

In [ ]:
vendor_sales_summary['VendorName'].unique()

In [ ]:
vendor_sales_summary['Description'].unique()

In [ ]:
vendor_sales_summary['Volume'] = vendor_sales_summary['Volume'].astype('float64')

In [ ]:
vendor_sales_summary.dtypes

In [ ]:
vendor_sales_summary.fillna(0, inplace = True)

In [ ]:
vendor_sales_summary.isnull().sum()

In [ ]:
vendor_sales_summary['VendorName'] = vendor_sales_summary['VendorName'].str.strip()

In [ ]:
vendor_sales_summary['VendorName'].unique()

In [ ]:
vendor_sales_summary['GrossProfit'] = vendor_sales_summary['TotalSalesDollars'] - vendor_sales_summary['TotalPurchaseDollars']

In [ ]:
import numpy as np

In [ ]:
vendor_sales_summary['GrossProfit'].min()

In [ ]:
vendor_sales_summary['ProfitMargin'] = (vendor_sales_summary['GrossProfit'] / vendor_sales_summary['TotalSalesDollars']) * 100

In [ ]:
vendor_sales_summary['StockTurnover'] = vendor_sales_summary['TotalSalesQuantity'] / vendor_sales_summary['TotalPurchaseQuantity']

In [ ]:
vendor_sales_summary['SalesPurchaseRatio'] = vendor_sales_summary['TotalSalesDollars'] / vendor_sales_summary['TotalPurchaseDollars']

In [ ]:
vendor_sales_summary

In [ ]:
vendor_sales_summary.columns

In [ ]:
cursor = conn.cursor()

In [ ]:
cursor.execute("""CREATE TABLE vendor_sales_summary (
    VendorNumber INT,
    VendorName VARCHAR(100),
    Brand INT,
    Description VARCHAR(100),
    PurchasePrice DECIMAL(10,2),
    ActualPrice DECIMAL (10,2),
    Volume INT,
    TotalPurchaseQuantity INT,
    TotalPurchaseDollars DECIMAl(10,2),
    TotalSalesQuantity INT,
    TotalSalesDollars DECIMAL(10,2),
    TotalSalesPrice DECIMAL(10,2),
    TotalExciseTax DECIMAL(10,2),
    FreightCost DECIMAL(15,2),
    GrossProfit DECIMAL(15,2),
    ProfitMargin DECIMAL(15,2),
    StockTurnover DECIMAL(15,2),
    SalesPurchaseRatio DECIMAL(15,2),
    PRIMARY KEY (VendorNumber, Brand)
);
""")
    

In [ ]:
pd.read_sql_query("SELECT * FROM vendor_sales_summary", conn)

In [ ]:
vendor_sales_summary.to_sql('vendor_sales_summary', conn, if_exists = 'replace', index = False)